In [1]:
import pandas as pd
import os
import sys
import yaml
import numpy as np
import requests
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '../base_code/'))
sys.path.append(parent_dir)
import query_db as qdb
import warnings
from basketball_rating_system import PlayerRatings
from scipy.stats import percentileofscore
import random
from functools import reduce
from scipy import stats
from datetime import date
from io import StringIO

In [2]:
warnings.filterwarnings('ignore')


with open('../base_code/primary_config.yaml', 'r') as file:
    config = yaml.safe_load(file)

conn_string = config['conn_string']

In [26]:
#class PlayerRatings:

    # def __init__(self, benchmark_data, scoring_data):
    #     self.benchmark_data = benchmark_data
    #     self.scoring_data = scoring_data
    #     self.key_metrics = ['points', 'assists']


    # def add_metrics(self):
    #     self.benchmark_data['assist_to_turnover_ratio'] = self.benchmark_data.apply(lambda row: self._assist_to_turnover_ratio(row), axis=1)
    #     self.scoring_data['assist_to_turnover_ratio'] = self.scoring_data.apply(lambda row: self._assist_to_turnover_ratio(row), axis=1)

    # def add_per_36_metrics(self):
    #     for col in self.key_metrics:
    #         new_col = '{}_36'.format(col)
    #         self.benchmark_data[new_col] = (self.benchmark_data[col]/self.benchmark_data['minutes_played']) * 36
    #         self.scoring_data[new_col] = (self.scoring_data[col]/self.scoring_data['minutes_played']) * 36

    # def _assist_to_turnover_ratio(self, row):
    #     asts = row['assists']
    #     tos = row['turnovers']
    #     if tos == 0:
    #         new_tos = 0.5
    #         final = asts/new_tos
    #     else:
    #         final = asts/tos
    #     return round(final, 4)

In [3]:
%%time

query = """
    select 
    *
    from basketball.player_game_box_scores


"""

benchmark = qdb.query_to_dataframe(query, conn_string)

benchmark['game_date'] = pd.to_datetime(benchmark.game_date).dt.date

sd = date(year=2025, month=8, day=8)

scoring = benchmark[benchmark.game_date >= sd]

pr = PlayerRatings(benchmark, scoring)

CPU times: user 90.3 ms, sys: 16.6 ms, total: 107 ms
Wall time: 252 ms


In [4]:
pr.add_per_36_metrics()

In [5]:
pr.add_metrics()

KeyError: 'player_field_goal_attempts'

In [ ]:
%%time

query = """
    select 
    distinct 
    pl
    from basketball.f_player_game_box_scores


"""

season_df = qdb.query_to_dataframe(query, conn_string)
